In [1]:
import pandas as pd
from IPython.display import display
import math
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Use the notebook to see a nicer display of the dataframes, particularly the heatmap

# Protein --> Pathway mapping
pathways = pd.read_csv('../../../Knowledge Graph/data/pw2proteins.csv')

# Disease Category --> PMID mapping
# Make a dictionary of Disease Name: [PMID,...PMID], ...
cat2pmid = {}
cat_name = pd.read_json('../../config/textcube_config.json')[0]
f_cat2pmidegory = open('../../data/entityfound_pmid2cell.txt')
i = 0
for line in f_cat2pmidegory:
    pmid = line.split('	')[0]
    cat = line.split('	')[1].split('\n')[0]
    i += 1

    if(pmid != 'doc_id'):
        cat_num = int(cat)
        if(cat_name[cat_num] not in cat2pmid.keys()):
            cat2pmid[cat_name[cat_num]] = []
        cat2pmid[cat_name[cat_num]].append(pmid)   


        
cat2protein = {}
cat2prot = {}
cat2pathway = {}
cat2pw = {}
for cat,v in cat2pmid.items():
    cat2protein[cat] = []
    cat2prot[cat] = []
    cat2pathway[cat] = []
    cat2pw[cat] = []

# Category: Proteins
f_entitycount = open('../../data/entitycount.txt')
for line in f_entitycount:
    pmid = line.split(' ')[0]
    protein_freq = line.split(' ')[1:]
    protein_list = []
    for prot in protein_freq:
        protein_list.append(prot.split('|'))
    
    for category,pmids in cat2pmid.items():
        if(pmid in cat2pmid[category]): 
            for protein in protein_list:
                cat2protein[category].append(protein)
    
for cat,prot_lists in cat2protein.items():
    curr_cat_prot_dict = {}
    for prot_list in prot_lists:
        protein = prot_list[0]
        freq = int(prot_list[1].split('\n')[0])
        if(protein not in curr_cat_prot_dict.keys()):
            curr_cat_prot_dict[protein] = 0
        curr_cat_prot_dict[protein] += freq
    cat2prot[cat] = curr_cat_prot_dict


# Category: Pathways
current = 0
for cat,protfreq_dicts in cat2prot.items():
    current += 1; print(cat, ':', current, 'of', len(cat2prot), end='\r')
    for protein,freq in protfreq_dicts.items():
        for ind in range(0,len(pathways)):
            if(pathways['protein'][ind] == protein):
                cat2pathway[cat].append({pathways['name'][ind]: freq})
                
for cat,pw_dicts in cat2pathway.items():
    curr_cat_pw_dict = {}
    for pw_dict in pw_dicts:
        pw = list(pw_dict.keys())[0]
        freq = int(list(pw_dict.values())[0])
        if(pw not in curr_cat_pw_dict.keys()):
            curr_cat_pw_dict[pw] = 0
        curr_cat_pw_dict[pw] += freq
    cat2pw[cat] = curr_cat_pw_dict
    
# Sort the pathway dictionary by the most popular pathway in each disease
sorted_cat2pw = {}
for cat in cat2pw:
    sorted_cat2pw[cat] = sorted(cat2pw[cat].items() , reverse=True, key=lambda x: x[1])
sorted_cat2pw


# Make dataframe ranking all pathways for each disease
top_pws = pd.DataFrame(columns = sorted_cat2pw.keys())
lengths = []
for cat in sorted_cat2pw: lengths.append(len(sorted_cat2pw[cat]))
max_cat = lengths.index(max(lengths))

for i in range(0,lengths[max_cat]):
    row = {}
    for cat in sorted_cat2pw:
        try: row.update({cat: sorted_cat2pw[cat][i][0]})
        except: row.update({'N/A':'N/A'})
    top_pws = top_pws.append(row, ignore_index=True)

top_pws.to_csv('Data/Popularity_Ranked_Pathways_in_Each_Category.csv')
display(top_pws)

# Make dataframe ranking all pathways for each disease
top_pws_counts = pd.DataFrame(columns = sorted_cat2pw.keys())
lengths = []
for cat in sorted_cat2pw: lengths.append(len(sorted_cat2pw[cat]))
max_cat = lengths.index(max(lengths))

for i in range(0,lengths[max_cat]):
    row = {}
    for cat in sorted_cat2pw:
        try: row.update({cat: float(sorted_cat2pw[cat][i][1])})
        except: row.update({'N/A':'N/A'})
    top_pws_counts = top_pws_counts.append(row, ignore_index=True)
    
display(top_pws_counts.style.background_gradient(cmap ='Reds')\
        .set_properties(**{'font-size': '10px'}))

top_pws_counts.to_csv('Data/Popularity_Ranked_Pathway_Counts_in_Each_Category.csv')

LHON  \
0                                           Metabolism   
1    The citric acid (TCA) cycle and respiratory el...   
2    Respiratory electron transport, ATP synthesis ...   
3                       Respiratory electron transport   
4                                 Complex I biogenesis   
..                                                 ...   
587                                                NaN   
588                                                NaN   
589                                                NaN   
590                                                NaN   
591                                                NaN   

                                                 MELAS  \
0                                           Metabolism   
1    The citric acid (TCA) cycle and respiratory el...   
2    Respiratory electron transport, ATP synthesis ...   
3                       Respiratory electron transport   
4                      Gene expression (Transcription)   
..                                                 ...   
587                                                NaN   
588                                                NaN   
589                                                NaN   
590                                                NaN   
591                                                NaN   

                                     ADOA  \
0                               Apoptosis   
1                   Programmed Cell Death   
2                 Regulation of Apoptosis   
3                Mitochondrial biogenesis   
4    Organelle biogenesis and maintenance   
..                                    ...   
587                                   NaN   
588                                   NaN   
589                                   NaN   
590                                   NaN   
591                                   NaN   

                                                 Leigh  \
0                                           Metabolism   
1    The citric acid (TCA) cycle and respiratory el...   
2    Respiratory electron transport, ATP synthesis ...   
3                       Respiratory electron transport   
4                      Gene expression (Transcription)   
..                                                 ...   
587                                                NaN   
588                                                NaN   
589                                                NaN   
590                                                NaN   
591                                                NaN   

                                     MERRF  \
0                               Metabolism   
1                   Metabolism of proteins   
2    Mitochondrial translation termination   
3                Mitochondrial translation   
4                              Translation   
..                                     ...   
587                                    NaN   
588                                    NaN   
589                                    NaN   
590                                    NaN   
591                                    NaN   

                                                   KSS  \
0                                           Metabolism   
1    The citric acid (TCA) cycle and respiratory el...   
2    Respiratory electron transport, ATP synthesis ...   
3                       Respiratory electron transport   
4                 Cellular response to chemical stress   
..                                                 ...   
587                                                NaN   
588                                                NaN   
589                                                NaN   
590                                                NaN   
591                                                NaN   

                                                Alpers  \
0                                           Metabolism   
1                                 Metabolism of lipi

/home/ubuntu/anaconda3/envs/caseolap/lib/python3.6/site-packages/matplotlib/colors.py:496: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [3]:
top_pws = pd.read_csv('Data/Popularity_Ranked_Pathways_in_Each_Category.csv')
top_pws.head(20)

Unnamed: 0                                               LHON  \
0            0                                         Metabolism   
1            1  The citric acid (TCA) cycle and respiratory el...   
2            2  Respiratory electron transport, ATP synthesis ...   
3            3                     Respiratory electron transport   
4            4                               Complex I biogenesis   
5            5                                          Apoptosis   
6            6                              Programmed Cell Death   
7            7                    Gene expression (Transcription)   
8            8                                      Immune System   
9            9                          Signaling by Interleukins   
10          10                Cytokine Signaling in Immune system   
11          11                      Generic Transcription Pathway   
12          12                    RNA Polymerase II Transcription   
13          13                           Mitochondrial biogenesis   
14          14               Organelle biogenesis and maintenance   
15          15  Gene and protein expression by JAK-STAT signal...   
16          16                           Interleukin-12 signaling   
17          17                    Interleukin-12 family signaling   
18          18               Cellular response to chemical stress   
19          19                       Cellular responses to stress   

                                                MELAS  \
0                                          Metabolism   
1   The citric acid (TCA) cycle and respiratory el...   
2   Respiratory electron transport, ATP synthesis ...   
3                      Respiratory electron transport   
4                     Gene expression (Transcription)   
5                       Generic Transcription Pathway   
6                     RNA Polymerase II Transcription   
7                        Cellular responses to stress   
8                       Cellular responses to stimuli   
9                  Transcriptional Regulation by TP53   
10               Cellular response to chemical stress   
11                           Mitochondrial biogenesis   
12               Organelle biogenesis and maintenance   
13                     TP53 Regulates Metabolic Genes   
14                            Cytoprotection by HMOX1   
15  Transcriptional activation of mitochondrial bi...   
16                    Intrinsic Pathway for Apoptosis   
17                                          Apoptosis   
18                              Programmed Cell Death   
19                                         Pyroptosis   

                                                 ADOA  \
0                                           Apoptosis   
1                               Programmed Cell Death   
2                             Regulation of Apoptosis   
3                            Mitochondrial biogenesis   
4                Organelle biogenesis and maintenance   
5                       PINK1-PRKN Mediated Mitophagy   
6                                           Mitophagy   
7                                 Selective autophagy   
8                                      Macroautophagy   
9                                           Autophagy   
10  Transcriptional activation of mitochondrial bi...   
11                                Signal Transduction   
12                                         Metabolism   
13                      Generic Transcription Pathway   
14                    Gene expression (Transcription)   
15                    RNA Polymerase II Transcription   
16                                         Hemostasis   
17                          Apoptotic execution phase   
18               Cellular response to chemical stress   
19                       Cellular responses to stress   

                                                Leigh  \
0                                          Metabolism   
1   The citric acid (TCA) cycle and respirat

In [29]:
nan = 0
total = 0
for i in range(0,len(top_pws)):
    for col in top_pws.columns[1:20]:
        if type(top_pws[col][i]) == float and math.isnan(top_pws[col][i]):
            nan += 1
        total += 1
print(nan, '/', total, ' = ', nan/total)

5305 / 11248  =  0.47163940256045517


In [44]:
pw_set = set()
for i in range(0,len(top_pws)):
    for col in top_pws.columns[1:20]:
        if type(top_pws[col][i]) != float:
            pw_set.add(top_pws[col][i])
        total += 1
print('Number of Pathways: ', len(pw_set))

Number of Pathways:  686


In [41]:
top_pws_counts = pd.read_csv('Data/Popularity_Ranked_Pathway_Counts_in_Each_Category.csv')
top_pws_counts.tail()

Unnamed: 0  LHON  MELAS  ADOA  Leigh  MERRF  KSS  Alpers  LBSL  PDD  ...  \
587         587   NaN    NaN   NaN    NaN    NaN  NaN     NaN   NaN  NaN  ...   
588         588   NaN    NaN   NaN    NaN    NaN  NaN     NaN   NaN  NaN  ...   
589         589   NaN    NaN   NaN    NaN    NaN  NaN     NaN   NaN  NaN  ...   
590         590   NaN    NaN   NaN    NaN    NaN  NaN     NaN   NaN  NaN  ...   
591         591   NaN    NaN   NaN    NaN    NaN  NaN     NaN   NaN  NaN  ...   

     MS  Parkin  ALS  AD  Epil  Schiz  FA  HSP  WD  N/A  
587 NaN     NaN  NaN NaN   1.0    NaN NaN  NaN NaN  NaN  
588 NaN     NaN  NaN NaN   1.0    NaN NaN  NaN NaN  NaN  
589 NaN     NaN  NaN NaN   1.0    NaN NaN  NaN NaN  NaN  
590 NaN     NaN  NaN NaN   1.0    NaN NaN  NaN NaN  NaN  
591 NaN     NaN  NaN NaN   1.0    NaN NaN  NaN NaN  NaN  

[5 rows x 21 columns]

In [51]:
total_sum = 0
total = 0
for i in range(0,len(top_pws_counts)):
    for col in top_pws_counts.columns[1:20]:
        if type(top_pws_counts[col][i]) == np.float64 and not math.isnan(top_pws_counts[col][i]):
            total_sum += top_pws_counts[col][i]
            total += 1
#Not as useful, the number of instances of pathways / number of pathways occurring in each disease (can count more than once if in more than one category)
#print(total_sum, '/', total, ' = ', total_sum/total)

1787463.0 / 5943  =  300.76779404341244


In [50]:
# Total instances of pathways appearing / pathways appearing = average number of instances a pathway occurs
print(total_sum, '/', len(pw_set), ' = ', total_sum/len(pw_set)) 

1787463.0 / 686  =  2605.631195335277


In [49]:
#Proteins appearing in an average of ___ disease categories
2897/729

3.9739368998628257